## 20. Attribute Descriptors
A descriptor is a class that implements a protocol consisting of the __get__, __set__, and __delete__ methods. Understanding descriptors is key to Python mastery. 

## LineItem Take #3: A Simple Descriptor
Descriptor class: A class implementing the descriptor protocol. <br>
Managed class: The class where the descriptor instances are declared as class attributes

In [2]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name

    def __set__(self, instance, value):
        ''' self is the descriptor instance, and instance is the managed instance.'''
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError("value must be > 0")


In [4]:
class LineItem:
    weight = Quantity("weight")
    price = Quantity("price")
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price


In [5]:
item = LineItem("toy", 2.5, 100)

In [6]:
item.__dict__

{'description': 'toy', 'weight': 2.5, 'price': 100}

In [8]:
LineItem.__dict__

mappingproxy({'__module__': '__main__',
              'weight': <__main__.Quantity at 0x1eb22c70470>,
              'price': <__main__.Quantity at 0x1eb22c70518>,
              '__init__': <function __main__.LineItem.__init__(self, description, weight, price)>,
              'subtotal': <function __main__.LineItem.subtotal(self)>,
              '__dict__': <attribute '__dict__' of 'LineItem' objects>,
              '__weakref__': <attribute '__weakref__' of 'LineItem' objects>,
              '__doc__': None})

In [9]:
import traceback

try:
    item.weight = 0
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-9-f23604991c7c>", line 4, in <module>
    item.weight = 0
  File "<ipython-input-2-31662b8935ca>", line 9, in __set__
    raise ValueError("value must be > 0")
ValueError: value must be > 0


## LineItem Take #4: Automatic Storage Attribute Names